In [11]:
%reload_ext autoreload
%autoreload 2

import glob
import os, gc
import numpy as numpy
import pandas as pd
import scipy as sp
import datatable as dt
from collections import defaultdict
from tqdm.notebook import tqdm
from sklearn.utils import shuffle
from sklearn.metrics import r2_score
from numba import njit
from utils import *

from IPython.display import clear_output

from sklearn.preprocessing import MinMaxScaler

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
# TF
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [12]:
N_FOLD = 10
N_MINS = 5
MIN_SIZE = 600 // N_MINS

SOL_NAME = '601-MLP'
DATA_NAME = '601'
mkdir(f'./models/{SOL_NAME}/')

In [13]:
# CONSTANT
MEAN = -5.762330803300896
STD = 0.6339307835941186
EPS = 1e-9

In [14]:
# get ids
list_stock_id = get_stock_id()
list_time_id = get_time_id()

# Functions

In [15]:
def transform_target(target):
    return (np.log(target + EPS) - MEAN) / STD

def inverse_target(target):
    return np.exp(MEAN + STD * target) - EPS

def np_rmspe(y_true, y_pred):
    y_true = inverse_target(y_true)
    y_pred = inverse_target(y_pred)
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def mspe_loss(y_true, y_pred):
    y_true = K.exp(MEAN + STD * y_true) - EPS
    y_pred = K.exp(MEAN + STD * y_pred) - EPS
    return K.sqrt(K.mean(K.square((y_true - y_pred) / y_true)))

In [16]:
def create_mlp(num_columns, num_labels, hidden_units, dropout_rates, learning_rate):
    inp = tf.keras.layers.Input(shape=(num_columns,))
    stock_inp = tf.keras.layers.Input(shape=(1,))
    stock_embedded = tf.keras.layers.Embedding(127, 24, input_length=1, name='stock_embedding')(stock_inp)
    stock_flattened = tf.keras.layers.Flatten()(stock_embedded)
    x = tf.keras.layers.Concatenate()([stock_flattened, inp])
    x = tf.keras.layers.BatchNormalization()(x)
    # x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        # x = tf.keras.layers.Dense(hidden_units[i], kernel_regularizer=tf.keras.regularizers.l1(dropout_rates))(x)
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        # x = tf.keras.layers.Dropout(dropout_rates[i+1])(x)
    # Output layer
    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation('linear')(x)
    model = tf.keras.models.Model(inputs=[stock_inp, inp], outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=mspe_loss,
    )
    return model

# Loading data

In [17]:
# train
df_train = dt.fread(f'./dataset/train_{DATA_NAME}_NN.csv').to_pandas()
fea_cols = [f for f in df_train if f not in ['time_id', 'target', 'pred_NN', 'stock_id', 'row_id']]

# result
df_result = dt.fread('./dataset/train.csv').to_pandas()
df_result = gen_row_id(df_result)

In [18]:
df_train['target'] = transform_target(df_train['target'])
df_train = gen_row_id(df_train)

# Evaluation

In [19]:
batch_size = 1024
hidden_units = [64, 32, 16]
dropout_rates = 0
learning_rate = 6e-3
epochs = 1000

list_seeds = [0, 11, 42, 777, 2045]

In [20]:
list_rmspe = []
for i_seed, seed in enumerate(list_seeds):
    df_train = add_time_fold(df_train, N_FOLD, seed=seed)
    list_rmspe += [[]]
    for i_fold in range(N_FOLD):
        gc.collect()
        df_tr = df_train.loc[df_train.fold!=i_fold]
        df_te = df_train.loc[df_train.fold==i_fold]

        S_train = df_tr['stock_id'].values
        X_train = df_tr[fea_cols].values
        y_train = df_tr[['target']].values
        S_test = df_te['stock_id'].values
        X_test = df_te[fea_cols].values
        y_test = df_te[['target']].values
        idx_test = df_train.loc[df_train.fold==i_fold].index
        print(f'Fold {i_seed+1}/{len(list_seeds)} | {i_fold+1}/{N_FOLD}', X_train.shape, X_test.shape)

        scaler = MinMaxScaler(feature_range=(-1, 1))         
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        save_pickle(scaler, f'./models/{SOL_NAME}/minmax_scaler_{i_seed}_{i_fold}.pkl')

        # Callbacks
        ckp_path = f'./models/{SOL_NAME}/model_{i_seed}_{i_fold}.hdf5'
        rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=8, min_delta=1e-5, verbose=2)
        es = EarlyStopping(monitor='val_loss', min_delta=1e-5, patience=28, restore_best_weights=True, verbose=2)

        model = create_mlp(X_train.shape[1], 1, hidden_units, dropout_rates, learning_rate) 
        history = model.fit([S_train, X_train], y_train, 
            epochs=epochs,
            validation_data=([S_test, X_test], y_test),
            validation_batch_size=len(y_test),
            batch_size=batch_size,
            verbose=2,
            callbacks=[rlr, es]
        )
        # model = tf.keras.models.load_model(ckp_path, custom_objects={'mspe_loss': mspe_loss})

        y_pred = model.predict([S_test, X_test], batch_size=len(y_test))
        curr_rmspe = np_rmspe(y_test, y_pred)
        list_rmspe[-1] += [curr_rmspe]
        model.save(ckp_path)
        # generate and save preds
        df_result.loc[idx_test, f'pred_{i_seed}'] = inverse_target(y_pred)
        clear_output()
        print(list_rmspe)

[[0.2065384341986309, 0.2173363191760012, 0.2112813695847935, 0.2077094683948919, 0.21202796593672618, 0.20708813480616417, 0.20808425230616037, 0.2109776844210437, 0.21223957865129364, 0.2232322952437671], [0.211102780277043, 0.21000414084425953, 0.2112115957794925, 0.2057211044816889, 0.20956488352831518, 0.21632720127164953, 0.2087688243806528, 0.20845586529544924, 0.22297633493933242, 0.21564741107640537], [0.21267757659547068, 0.21096362557209708, 0.20821285795550049, 0.21313546539072203, 0.21011651408211687, 0.21379642068928925, 0.20637897928746365, 0.20374189533433198, 0.21619579655628884, 0.21756747123301495], [0.2105981847608589, 0.21369209105083814, 0.20753003489652366, 0.21333074745592226, 0.209805174170137, 0.2105325790247582, 0.21528117915212838, 0.2074926914328247, 0.21468802081786656, 0.21171095040868396], [0.20455991809808982, 0.20498581871446755, 0.20718383851194105, 0.21710600354303639, 0.2133529851894719, 0.21568096064182532, 0.21347158762924442, 0.20810158581370172,

In [21]:
df_result.to_csv(f'./results/{SOL_NAME}.csv', index=False)

In [23]:
for i in range(len(list_seeds)):
    print(i, rmspe(df_result['target'], df_result[f'pred_{i}']))
print('All: ', rmspe(df_result['target'], df_result[[f'pred_{i}' for i in range(len(list_seeds))]].mean(axis=1)))

0 0.21170891020200328
1 0.21203142915951242
2 0.21131821622607144
3 0.2114825189040702
4 0.21128732061052158
All:  0.20826322504230063
